In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('recommender').getOrCreate()
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import TrainValidationSplit, ParamGridBuilder

In [2]:
# data = spark.read.csv('data/reviews/all_reviews.csv', inferSchema=True, header=True).rdd


In [2]:
from pyspark.sql import Row

lines = spark.read.text("data/reviews/all_reviews.txt").rdd
parts = lines.map(lambda row: row.value.split(":"))
ratingsRDD = parts.map(lambda p: Row(userId=int(p[0]), itemId=int(p[1]),
                                     rating=float(p[2])))
ratings = spark.createDataFrame(ratingsRDD)

In [3]:
ratings.show(5)

+------+------+------+
|itemId|rating|userId|
+------+------+------+
|    19|   3.0|  2976|
|    19|   5.0|  3339|
|    19|   4.0|  1472|
|    19|   5.0|   603|
|    19|   5.0|   332|
+------+------+------+
only showing top 5 rows



In [4]:
(training, test) = ratings.randomSplit([0.8, 0.2])

In [5]:
# training.orderBy(training.userId.desc()).show(5)

In [5]:
# Build the recommendation model using ALS on the training data
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
als = ALS(userCol="userId", itemCol="itemId", ratingCol="rating", nonnegative=True,
          coldStartStrategy="drop")
model = als.fit(training)

# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))


Root-mean-square error = 1.2517839266497903


In [1]:

# Generate top 10 movie recommendations for each user
userRecs = model.recommendForAllUsers(10)
# Generate top 10 user recommendations for each movie
itemRecs = model.recommendForAllItems(10)

# # Generate top 10 movie recommendations for a specified set of users
# users = ratings.select(als.getUserCol()).distinct().limit(3)
# userSubsetRecs = model.recommendForUserSubset(users, 10)
# # Generate top 10 user recommendations for a specified set of movies
# movies = ratings.select(als.getItemCol()).distinct().limit(3)
# movieSubSetRecs = model.recommendForItemSubset(movies, 10)

NameError: name 'model' is not defined